Parsing data from https://apnews.com/technology

In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import random

### 1. Same tytuły - podejście próbne

In [6]:
url = "https://apnews.com/technology"
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

titles = soup.find_all("span", class_="PagePromoContentIcons-text")

for t in titles:
    title = t.get_text(strip=True)
    print(title)

The Associated Press is an independent global news organization dedicated to factual reporting. Founded in 1846, AP today remains the most trusted source of fast, accurate, unbiased news in all formats and the essential provider of the technology and services vital to the news business. More than half the world’s population sees AP journalism every day.

The Associated Press is an independent global news organization dedicated to factual reporting. Founded in 1846, AP today remains the most trusted source of fast, accurate, unbiased news in all formats and the essential provider of the technology and services vital to the news business. More than half the world’s population sees AP journalism every day.

Republican enthusiasm for Musk cools after his feud with Trump, a new AP-NORC poll finds
A new poll finds Elon Musk has lost some of his luster with Republicans since his public falling-out with President Donald Trump.
Denmark tests unmanned robotic sailboat fleet with tensions high in

### 2. pierwsze 1000 znaków artykułów, imo działa

In [10]:
headers = {
    "User-Agent": "Mozilla/5.0"
}

base_url = "https://apnews.com"
section_url = f"{base_url}/technology"

# Pobierz stronę główną sekcji
response = requests.get(section_url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Znajdź tytuły i linki
titles = soup.find_all("span", class_="PagePromoContentIcons-text")

articles = []

for t in titles[:10]:  # tylko 10 pierwszych artykułów
    a_tag = t.find_parent("a")
    title = t.get_text(strip=True)
    href = a_tag['href'] if a_tag else None
    if href and not href.startswith("http"):
        href = base_url + href

    if href:
        try:
            # Losowe opóźnienie
            time.sleep(random.uniform(1, 3))

            article_resp = requests.get(href, headers=headers)
            article_soup = BeautifulSoup(article_resp.text, "html.parser")

            # Szukamy tylko w treści artykułu
            content_div = article_soup.find("div", class_="RichTextStoryBody")
            if content_div:
                paragraphs = content_div.find_all("p")
                content = "\n".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
            else:
                content = "(brak treści)"

            articles.append({
                "title": title,
                "url": href,
                "content": content
            })

            print(f"=== {title} ===")
            print(f"URL: {href}")
            print(content[:1000], "\n")  # pierwsze 500 znaków
            print("-" * 80)

        except Exception as e:
            print(f"Błąd przy pobieraniu {href}: {e}")

=== Republican enthusiasm for Musk cools after his feud with Trump, a new AP-NORC poll finds ===
URL: https://apnews.com/article/musk-poll-trump-tesla-doge-tax-bill-e95e9b4d26f66838e13d0d5169e402b8
WASHINGTON (AP) — Tech billionaire Elon Musk has lost some of his luster with Republicans since hismessy public falling-outwith President Donald Trump last week, a new survey finds.
Fewer Republicans view Trump’s onetime government efficiency bulldog “very favorably”compared with April, according to the poll fromThe Associated Press-NORC Center for Public Affairs Research.
Though most Republicans continue to hold a positive view of Musk, their diminished fervor suggests hisvocal oppositionto Trump’s signature spending and tax cut legislation — and Musk’ssubsequent online political and personal taunts— may have cost him some enthusiasm within the party.
“Some things have happened lately that have changed how I feel about him a little,” said Alabama Republican Katye Long, whose feelings for Mu